In [1]:
rm(list = ls())

In [2]:
# libraries
suppressMessages(library(tidyverse))
library(broom)

In [3]:
Data <- read.csv("/Users/mpaga/Downloads/train.csv",sep=",")

In [4]:
dim(Data)

[1] 1460   81

In [5]:
names(Data)

 [1] "Id"            "MSSubClass"    "MSZoning"      "LotFrontage"  
 [5] "LotArea"       "Street"        "Alley"         "LotShape"     
 [9] "LandContour"   "Utilities"     "LotConfig"     "LandSlope"    
[13] "Neighborhood"  "Condition1"    "Condition2"    "BldgType"     
[17] "HouseStyle"    "OverallQual"   "OverallCond"   "YearBuilt"    
[21] "YearRemodAdd"  "RoofStyle"     "RoofMatl"      "Exterior1st"  
[25] "Exterior2nd"   "MasVnrType"    "MasVnrArea"    "ExterQual"    
[29] "ExterCond"     "Foundation"    "BsmtQual"      "BsmtCond"     
[33] "BsmtExposure"  "BsmtFinType1"  "BsmtFinSF1"    "BsmtFinType2" 
[37] "BsmtFinSF2"    "BsmtUnfSF"     "TotalBsmtSF"   "Heating"      
[41] "HeatingQC"     "CentralAir"    "Electrical"    "X1stFlrSF"    
[45] "X2ndFlrSF"     "LowQualFinSF"  "GrLivArea"     "BsmtFullBath" 
[49] "BsmtHalfBath"  "FullBath"      "HalfBath"      "BedroomAbvGr" 
[53] "KitchenAbvGr"  "KitchenQual"   "TotRmsAbvGrd"  "Functional"   
[57] "Fireplaces"    "FireplaceQu"

In [6]:
#remove Id colum
Data["Id"] <- NULL
dim(Data)

[1] 1460   80

In [7]:
hist(Data$SalePrice)

In [8]:
summary(Data$SalePrice)

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
  34900  129975  163000  180921  214000  755000 

In [9]:
skimr::skim_without_charts(Data,where(is.numeric))-> num_skimData

In [10]:
(numVal_na <- num_skimData$skim_variable[num_skimData$n_missing !=0] )

[1] "LotFrontage" "MasVnrArea"  "GarageYrBlt"

In [11]:
mean(is.na(Data$LotFrontage))

[1] 0.1773973

In [12]:
mean(is.na(Data$MasVnrType))

[1] 0.005479452

In [13]:
mean(is.na(Data$GarageYrBlt))

[1] 0.05547945

In [14]:
skimr::skim_without_charts(Data,where(is.character))-> cat_skimData

In [15]:
summary(cat_skimData)

── Data Summary ────────────────────────
                           Values
Name                       Data  
Number of rows             1460  
Number of columns          80    
_______________________          
Column type frequency:           
  character                43    
________________________         
Group variables            None  

In [16]:
cat_skimData$skim_variable[cat_skimData$n_missing !=0]

 [1] "Alley"        "MasVnrType"   "BsmtQual"     "BsmtCond"    
 [5] "BsmtExposure" "BsmtFinType1" "BsmtFinType2" "Electrical"  
 [9] "FireplaceQu"  "GarageType"   "GarageFinish" "GarageQual"  
[13] "GarageCond"   "PoolQC"       "Fence"        "MiscFeature" 

Let's impute some of these numerical features

In [17]:
numVal_na

[1] "LotFrontage" "MasVnrArea"  "GarageYrBlt"

In [18]:
colMeans(is.na(Data[numVal_na]))

LotFrontage  MasVnrArea GarageYrBlt 
0.177397260 0.005479452 0.055479452 

In [19]:
# list of num featute to impute
imputeVal_list <- apply(Data[numVal_na],2, FUN = "median",na.rm = T,simplify = list)

In [20]:
#impute numerical features 
Data[numVal_na] <- replace_na(Data[numVal_na] ,
                              replace = imputeVal_list
)

In [21]:
# check  na
colMeans(is.na(Data[numVal_na]))

LotFrontage  MasVnrArea GarageYrBlt 
          0           0           0 

In [22]:
Data |> 
  select_if(is.numeric) |> 
  unique() |> 
  dim()

[1] 1460   37

to be continued !